<a href="https://colab.research.google.com/github/406programmer/DATA-SCIENCE-PROGRAMMING-ASSIGNMENT/blob/main/ASSOCIATION_RULE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=5571f6ba9026fa4308c4453255e86f283269c260131b47d27366bd36a9e381c6
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


# How to Mine Association Rules Using R/Python

In [ ]:
import pandas as pd
from apyori import apriori

# Load the bank dataset (replace 'bank.csv' with the actual path to your dataset)
bank_data = pd.read_csv('/content/bank.csv')

# Subset the data to include only the relevant columns
min_bank = bank_data[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome', 'deposit']]

# Print column names to check for correctness
print(min_bank.columns)

# Convert categorical variables to string for apyori
min_bank = min_bank.astype(str)

# Check if "CustServ Calls" is in the column names
if 'CustServ Calls' in min_bank.columns:
    # Convert Customer Service Calls to a factor
    min_bank['CustServ Calls'] = pd.Categorical(min_bank['CustServ Calls'], ordered=True)

    # Obtain baseline distribution information
    baseline_tables = []
    for column in min_bank.columns:
        t = pd.crosstab(min_bank[column], margins=True, margins_name='Total', normalize='index')
        baseline_tables.append(t)

    # Install and load the apyori package (if not already installed)
    # !pip install apyori
    from apyori import apriori

    # Flatten the tables for apyori
    transactions = []
    for t in baseline_tables:
        transactions.append([(str(item),) for item in t.index if item != 'Total'])

    # Run apriori algorithm
    rules = apriori(transactions, min_support=0.01, min_confidence=0.4, min_lift=1, min_length=2)

    # Convert apyori results to a dataframe for further analysis
    rules_df = pd.DataFrame(list(rules))

    # Filter out rules containing 'deposit' in antecedent
    filtered_rules_df = rules_df[~rules_df['items'].astype(str).str.contains('deposit')]

    # Display the top 10 rules sorted by lift
    top_rules = filtered_rules_df.sort_values(by='lift', ascending=False).head(10)
    print(top_rules)
else:
    print("Column 'CustServ Calls' not found in the dataset.")


Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'poutcome', 'deposit'],
      dtype='object')
Column 'CustServ Calls' not found in the dataset.


# How to Apply the Confidence Quotient Criterion Using Python

In [ ]:
import pandas as pd
from apyori import apriori

# Load the bank dataset (replace 'bank.csv' with the actual path to your dataset)
bank_data = pd.read_csv('/content/bank.csv')

# Subset the data to include only the relevant columns
min_bank = bank_data[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome', 'deposit']]

# Convert categorical variables to string for apyori
min_bank = min_bank.astype(str)

# Flatten the tables for apyori
transactions = []
for column in min_bank.columns:
    transactions.append([(str(item),) for item in pd.unique(min_bank[column])])

# Run apriori algorithm with Confidence Quotient criterion
rules_confquot = apriori(transactions, min_support=0.01, min_confidence=0.05, min_lift=1, min_length=2, max_length=2,
                        arem="quot", aval=True, minval=0.4, target="rules")

# Convert apyori results to a dataframe for further analysis
rules_confquot_df = pd.DataFrame(list(rules_confquot))

# Filter out rules containing 'deposit' in antecedent
filtered_rules_confquot_df = rules_confquot_df[~rules_confquot_df['items'].astype(str).str.contains("deposit")]

# Display the rules
print(filtered_rules_confquot_df)


                              items   support  \
0                       ((admin.,))  0.111111   
1                  ((blue-collar,))  0.111111   
2                     ((cellular,))  0.111111   
3                     ((divorced,))  0.111111   
4                 ((entrepreneur,))  0.111111   
..                              ...       ...   
105  ((technician,), (unemployed,))  0.111111   
106     ((technician,), (unknown,))  0.111111   
107      ((unknown,), (telephone,))  0.111111   
108       ((unknown,), (tertiary,))  0.111111   
109     ((unknown,), (unemployed,))  0.111111   

                                    ordered_statistics  
0       [((), (('admin.',)), 0.1111111111111111, 1.0)]  
1    [((), (('blue-collar',)), 0.1111111111111111, ...  
2     [((), (('cellular',)), 0.1111111111111111, 1.0)]  
3     [((), (('divorced',)), 0.1111111111111111, 1.0)]  
4    [((), (('entrepreneur',)), 0.1111111111111111,...  
..                                                 ...  
105  [((), (

# How to Apply the Confidence Difference Criterion Using Python

In [ ]:
import pandas as pd
from apyori import apriori

# Load the bank dataset (replace 'bank.csv' with the actual path to your dataset)
bank_data = pd.read_csv('/content/bank.csv')

# Subset the data to include only the relevant columns
min_bank = bank_data[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome', 'deposit']]

# Convert categorical variables to string for apyori
min_bank = min_bank.astype(str)

# Flatten the tables for apyori
transactions = []
for column in min_bank.columns:
    transactions.append([(str(item),) for item in pd.unique(min_bank[column])])

# Run apriori algorithm with confidence difference criterion
rules_confdiff = apriori(transactions, min_support=0.01, min_confidence=0.4, min_lift=1, min_length=2)

# Display all rules before filtering
for rule in rules_confdiff:
    print(rule)

# Filter rules based on confidence difference criterion
confidence_difference_threshold = 0.1  # Set your desired confidence difference threshold
filtered_rules_confdiff = [
    rule for rule in rules_confdiff
    if any(
        abs(stat.confidence - rule.ordered_statistics[i].confidence) >= confidence_difference_threshold
        for i, stat in enumerate(rule.ordered_statistics)
    )
]

# Display the filtered rules
for rule in filtered_rules_confdiff[:10]:
    print(rule)
